NAME : BALKARAN SINGH BATCH : 3Q11 ROLL NO : 102317009

ASSIGNMENT - 4 [UPDATED]

In [1]:
!pip install requests beautifulsoup4 selenium pandas lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


Q1: Books Scraping

In [2]:
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
books = []
page = 1
while True:
    url=base_url.format(page)
    response=requests.get(url)
    if response.status_code != 200:
        break
    soup=BeautifulSoup(response.text, "lxml")
    articles=soup.find_all("article", class_="product_pod")
    if not articles:
        break
    for article in articles:
        title=article.h3.a["title"]
        price=article.find("p", class_="price_color").text.strip()
        availability=article.find("p", class_="instock availability").text.strip()
        rating=article.p["class"][1]
        books.append([title, price, availability, rating])

    page+=1
books_df=pd.DataFrame(books, columns=["Title", "Price", "Availability", "Star Rating"])
books_df.to_csv("books.csv",index=False)
books_df.head()

,Title,Price,Availability,Star Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


Q2: IMDB Top 250

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

browser=webdriver.Chrome(options=chrome_opts)

browser.get("https://www.imdb.com/chart/top/")
time.sleep(5)
film_list=[]
movie_cards=browser.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")

for rank, card in enumerate(movie_cards, start=1):
    try:
        name=card.find_element(By.CSS_SELECTOR,"h3").text
        release_year=card.find_element(By.CSS_SELECTOR,".cli-title-metadata-item").text
        score=card.find_element(By.CSS_SELECTOR,".ipc-rating-star--imdb").text.split()[0]
        film_list.append([rank,name,release_year,score])
    except Exception as e:
        print(f"Skipping a card due to error: {e}")

browser.quit()

imdb_table = pd.DataFrame(film_list, columns=["Position", "Movie", "Release Year", "Rating"])
imdb_table.to_csv("imdb_top250.csv", index=False)
print(imdb_table.head())

   Position                        Movie Release Year Rating
0         1  1. The Shawshank Redemption         1994    9.3
1         2             2. The Godfather         1972    9.2
2         3           3. The Dark Knight         2008    9.1
3         4     4. The Godfather Part II         1974    9.0
4         5              5. 12 Angry Men         1957    9.0


Q3: Weather Scraping

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

browser=webdriver.Chrome(options=chrome_opts)

browser.get("https://www.timeanddate.com/weather/")
time.sleep(5)

rows=browser.find_elements(By.CSS_SELECTOR,"table tbody tr")

weather_list=[]
for row in rows[:20]:
    try:
        city=row.find_element(By.CSS_SELECTOR,"a").text
        temp=row.find_elements(By.CSS_SELECTOR,"td")[1].text
        condition=row.find_elements(By.CSS_SELECTOR,"td")[2].text
        weather_list.append([city, temp, condition])
    except Exception as e:
        print(f"Skipping row due to error:{e}")

browser.quit()

weather_table=pd.DataFrame(weather_list, columns=["City","Temp","Cond."])
weather_table.to_csv("weather.csv", index=False)
print(weather_table.head())

          City          Temp Cond.
0        Accra   Tue 7:28 am      
1  Addis Ababa  Tue 10:28 am      
2     Adelaide   Tue 4:58 pm      
3      Algiers   Tue 8:28 am      
4       Almaty  Tue 12:28 pm      
